In [1]:
from huggingface_hub import notebook_login

/home/kit/anaconda3/envs/tf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#/========== PATHS and VARIABLES ==========
model_id = "google/vit-base-patch16-224-in21k"
dataset_folder = "EuroSAT/2750"


In [3]:
#/========== DATASET ==========
import createDataset
import datasets
import os
import PIL

dataset = createDataset.create_image_folder_dataset(dataset_folder)
img_class_labels = dataset.features["label"].names

def create_image_folder_dataset(root_path):
    ## list directories
    _CLASS_NAMES = os.listdir(root_path)
    ## enumerate features
    features=datasets.Features({
        "img": datasets.Image(),
        "label": datasets.features.ClassLabel(names=_CLASS_NAMES),
    })
    ## create/populate list of datapoints
    img_data_files=[]
    label_data_files=[]
    for img_class in os.listdir(root_path):
        for img in os.listdir(os.path.join(root_path, img_class)):
            path_ = os.path.join(root_path, img_class, img)
            img_data_files.append(path_)
            label_data_files.append(img_class)
    ## create dataset
    return datasets.Dataset.from_dict({"img":img_data_files, "label":label_data_files}, features=features)

## Make eurosat dataset
eurosat_ds = create_image_folder_dataset("EuroSAT/2750")
img_class_labels = eurosat_ds.features["label"].names

In [4]:
import tensorflow as tf
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

#/========== PREPROCESSING ==========
from transformers import ViTImageProcessor
from tensorflow import keras
from keras import layers

#feature_extractor = ViTFeatureExtractor.from_pretrained(model_id)
image_processor = ViTImageProcessor.from_pretrained(model_id)

data_augmentation = tf.keras.Sequential(
    [
        layers.Resizing(image_processor.size, image_processor.size),
        layers.Rescaling(1./255),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="data_augmentation",
)
# Use keras to augment the image data: apply random transforms to the data to allow the model to generalise better.
def augmentation(examples):
    examples["pixel_values"] = [data_augmentation(image) for image in examples["img"]]
    return examples

# Basic image processing
def process(examples):
    examples.update(image_processor(examples['img'], ))
    return examples

## Process the dataset
processed_dataset = eurosat_ds.map(process, batched=True)
processed_dataset

## create test dataset 
# test size will be 30% of train dataset
test_size=.3
processed_dataset = processed_dataset.shuffle().train_test_split(test_size=test_size)

2024-06-11 16:48:57.431037: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 16:48:57.451485: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 16:48:59.080049: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13898 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9
Map: 100%|████████████████| 27000/27000 [00:40<00:00, 669.40 examples/s]


In [5]:
#/========== FINETUNING ==========
from huggingface_hub import HfFolder
import tensorflow as tf

id2label = {str(i): label for i, label in enumerate(img_class_labels)}
label2id = {v: k for k, v in id2label.items()}

# set hyperparameters
num_train_epochs = 5
train_batch_size = 32
eval_batch_size = 32
learning_rate = 3e-5
weight_decay_rate = 0.01
num_warmup_steps = 0
output_dir=model_id.split("/")[1]
hub_token = HfFolder.get_token()
hub_model_id = f'{model_id.split("/")[1]}-euroSat'
fp16=True
# when training on floating-point 16 set GPU policy to accomodate floating points
if fp16:
    tf.keras.mixed_precision.set_global_policy("mixed_float16")


In [6]:
#/=========== Convert dataset to Tensorflow dataset ===========
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

# convert train dataset to tf.data.Dataset
tf_train_dataset = processed_dataset["train"].to_tf_dataset(
    columns=["pixel_values"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=train_batch_size,
    collate_fn=data_collator
)

# convert test dataset to tf.data.Dataset
tf_eval_dataset = processed_dataset["test"].to_tf_dataset(
    columns=["pixel_values"],
    label_cols=["labels"],
    shuffle=True,
    batch_size=train_batch_size,
    collate_fn=data_collator
)

/home/kit/anaconda3/envs/tf/lib/python3.12/site-packages/datasets/arrow_dataset.py:410: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [15]:
#/========== Prepare the MODEL ==========
from transformers import TFViTForImageClassification, create_optimizer
from datasets import load_metric
import tensorflow as tf
 
# create optimizer wight weigh decay
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=num_warmup_steps,
)
 
# load pre-trained ViT model
model = TFViTForImageClassification.from_pretrained(
    model_id,
    num_labels=len(img_class_labels),
    id2label=id2label,
    label2id=label2id,
)
 
# define loss
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
 
# define metrics
#metrics=[
#    keras.metrics.SparseCategoricalAccuracy(),
#    keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="top-3-accuracy"),
#]

metric = load_metric("accuracy")

 
# compile model
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'],
              )

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/kit/anaconda3/envs/tf/lib/python3.12/site-packages/datasets/load

In [16]:
#/========== FIT MODEL ==========
train_results = model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    #callbacks=callbacks,
    epochs=num_train_epochs,
)

Epoch 1/5


W0000 00:00:1718150153.628602   15899 asm_compiler.cc:252] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
W0000 00:00:1718150153.628626   15899 asm_compiler.cc:255] Used ptxas at /usr/bin/ptxas
2024-06-11 16:55:53.628666: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-06-11 16:55:53.628681: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-06-11 16:55:53.628689: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-06-11 16:55:53.628696: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to c

  1/591 [..............................] - ETA: 2:27:51 - loss: 2.3230 - accuracy: 0.0312

2024-06-11 16:55:54.349923: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-06-11 16:55:54.371183: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-06-11 16:55:54.390196: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.


591/591 [==============================] - 217s 343ms/step - loss: 0.5357 - accuracy: 0.9420 - val_loss: 0.2075 - val_accuracy: 0.9695
Epoch 2/5
591/591 [==============================] - 202s 342ms/step - loss: 0.1234 - accuracy: 0.9869 - val_loss: 0.1286 - val_accuracy: 0.9772
Epoch 3/5
591/591 [==============================] - 202s 342ms/step - loss: 0.0652 - accuracy: 0.9930 - val_loss: 0.0791 - val_accuracy: 0.9856
Epoch 4/5
591/591 [==============================] - 202s 342ms/step - loss: 0.0391 - accuracy: 0.9973 - val_loss: 0.0665 - val_accuracy: 0.9877
Epoch 5/5
591/591 [==============================] - 202s 342ms/step - loss: 0.0286 - accuracy: 0.9990 - val_loss: 0.0601 - val_accuracy: 0.9893


In [18]:
#/========== SAVE MODEL ==========
model.save("euroSAT.keras")

/home/kit/anaconda3/envs/tf/lib/python3.12/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [17]:
#/========== EVALUATE MODEL ==========
model.evaluate(tf_eval_dataset)

254/254 [==============================] - 58s 227ms/step - loss: 0.0601 - accuracy: 0.9893


[0.060085661709308624, 0.9892592430114746]

In [ ]:
#/========== Predict label ==========
probs = model.predict(tf_eval_dataset)
#classes = keras.np_utils.probas_to_classes(probs)

254/254 [==============================] - 58s 219ms/step
